In [1]:
import pandas as pd
from tensorflow.contrib.keras.api.keras.models import Sequential
from tensorflow.contrib.keras.api.keras.layers import Dense,Dropout
from tensorflow.contrib.keras.api.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.contrib.keras.api.keras import backend

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV


/home/userhadoop/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
data = pd.read_csv('Churn_Modelling.csv')

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
y= data['Exited']

In [6]:
data.drop(['RowNumber','CustomerId','Surname',],axis=1,inplace=True)

In [7]:
x= pd.get_dummies(data,['Geography','Gender'],drop_first=True)

In [8]:
x.drop('Exited',axis=1,inplace=True)


In [9]:
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.33, random_state=42)

In [10]:
scaler =StandardScaler()

In [11]:
X_train= scaler.fit_transform(X_train)
X_test =scaler.fit_transform(X_test)

In [12]:
def build_ann():
    classifier = Sequential()
    classifier.add(Dense(kernel_initializer='uniform',input_dim=11,activation='relu',units=6))
    classifier.add(Dropout(rate=0.1))
    classifier.add(Dense(kernel_initializer='uniform',activation='relu',units=6))
    classifier.add(Dense(kernel_initializer='uniform',activation='sigmoid',units=1))
    classifier.compile(optimizer='adam',loss= 'binary_crossentropy',metrics=['accuracy'])
    return classifier

In [13]:
classifier_eval = KerasClassifier(build_fn=build_ann,batch_size=10,epochs=10)

In [14]:
accuracy= cross_val_score(estimator=classifier_eval,X=X_train,y=y_train,cv=5,n_jobs=-1)

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
5360/5360 [==============================] - 4s 699us/step - loss: 2.3159 - acc: 0.5297
Epoch 2/10
5360/5360 [==============================] - 4s 685us/step - loss: 0.8686 - acc: 0.6666
Epoch 2/10
5360/5360 [==============================] - 3s 553us/step - loss: 0.5600 - acc: 0.7791
Epoch 3/10
Epoch 3/10
5360/5360 [==============================] - 3s 618us/step - loss: 0.5373 - acc: 0.7942
Epoch 3/10
5360/5360 [==============================] - 3s 549us/step - loss: 0.5259 - acc: 0.7860
Epoch 4/10
5360/5360 [==============================] - 3s 524us/step - loss: 0.5160 - acc: 0.7931
Epoch 4/10
5360/5360 [==============================] - 3s 567us/step - loss: 0.4852 - acc: 0.7948
Epoch 4/10
5360/5360 [==============================] - 3s 554us/step - loss: 0.4703 - acc: 0.7950
Epoch 5/10
5360/5360 [==============================] - 3s 579us/step - loss: 0.4627 - acc: 0.7955
Epoch 6/10
5360/5360 [==============================] - 3s 581us/

In [15]:
accuracy.mean()

0.8153731302094103

In [16]:
accuracy.std()

0.012295101591851335

In [27]:
accuracy

array([0.80820895, 0.80895522, 0.83955223, 0.80671641, 0.81343283])

In [22]:
def build_fn(optimizer):
    classifier = Sequential()
    classifier.add(Dense(kernel_initializer='uniform',input_dim=11,activation='relu',units=6))
    classifier.add(Dropout(rate=0.1))
    classifier.add(Dense(kernel_initializer='uniform',activation='relu',units=6))
    classifier.add(Dense(kernel_initializer='uniform',activation='sigmoid',units=1))
    classifier.compile(optimizer,loss= 'binary_crossentropy',metrics=['accuracy'])
    return classifier

In [23]:
classifier_optimizer = KerasClassifier(build_fn)

In [24]:
parameter ={'optimizer':['adam'],'batch_size':[20,30],'epochs':[5,10]}

In [25]:
grid_search = GridSearchCV(estimator=classifier_optimizer,param_grid=parameter,scoring='accuracy',cv=5, n_jobs=-1,verbose=2)

In [26]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] optimizer=adam, batch_size=20, epochs=5 .........................
[CV] optimizer=adam, batch_size=20, epochs=5 .........................
[CV] optimizer=adam, batch_size=20, epochs=5 .........................
[CV] optimizer=adam, batch_size=20, epochs=5 .........................
Epoch 1/5
Epoch 1/5
Epoch 1/5
Epoch 1/5
5360/5360 [==============================] - 2s 364us/step - loss: 2.8850 - acc: 0.4159
Epoch 2/5
5360/5360 [==============================] - 2s 399us/step - loss: 2.4436 - acc: 0.4420
Epoch 2/5
5360/5360 [==============================] - 2s 403us/step - loss: 2.1796 - acc: 0.4672
Epoch 2/5
5360/5360 [==============================] - 2s 393us/step - loss: 1.6806 - acc: 0.5118
Epoch 2/5
5360/5360 [==============================] - 2s 308us/step - loss: 0.7415 - acc: 0.6713
Epoch 3/5
5360/5360 [==============================] - 1s 279us/step - loss: 0.6460 - acc: 0.7285
Epoch 3/5
5360/5360 [=================

5360/5360 [==============================] - 2s 293us/step - loss: 0.4717 - acc: 0.7968
Epoch 10/10
5360/5360 [==============================] - 1s 266us/step - loss: 0.5373 - acc: 0.7897
Epoch 4/10
1720/5360 [========>.....................] - ETA: 0s - loss: 0.5228 - acc: 0.7930[CV] ......... optimizer=adam, batch_size=20, epochs=10, total=  19.8s
[CV] optimizer=adam, batch_size=20, epochs=10 ........................
3780/5360 [====================>.........] - ETA: 0s - loss: 0.5191 - acc: 0.7921[CV] ......... optimizer=adam, batch_size=20, epochs=10, total=  20.1s
[CV] optimizer=adam, batch_size=30, epochs=5 .........................
5360/5360 [==============================] - 1s 257us/step - loss: 0.5201 - acc: 0.7897
Epoch 5/10
5360/5360 [==============================] - 2s 307us/step - loss: 0.5018 - acc: 0.7944
Epoch 6/10
5360/5360 [==============================] - 2s 455us/step - loss: 3.8652 - acc: 0.4293
Epoch 2/5
5360/5360 [==============================] - 1s 255us/step 

3510/5360 [==================>...........] - ETA: 0s - loss: 0.5235 - acc: 0.7872[CV] ......... optimizer=adam, batch_size=20, epochs=10, total=  20.0s
[CV] optimizer=adam, batch_size=30, epochs=10 ........................
5360/5360 [==============================] - 1s 223us/step - loss: 0.5189 - acc: 0.7901
Epoch 1/10
[CV] .......... optimizer=adam, batch_size=30, epochs=5, total=   7.7s
[CV] optimizer=adam, batch_size=30, epochs=10 ........................
5360/5360 [==============================] - 2s 380us/step - loss: 3.7212 - acc: 0.4121
Epoch 2/10
5360/5360 [==============================] - 1s 146us/step - loss: 1.3734 - acc: 0.5375
Epoch 3/10
5360/5360 [==============================] - 1s 149us/step - loss: 0.5965 - acc: 0.7821
Epoch 4/10
5360/5360 [==============================] - 2s 344us/step - loss: 2.0868 - acc: 0.4271
Epoch 2/10
5360/5360 [==============================] - 1s 157us/step - loss: 1.0993 - acc: 0.5362
Epoch 3/10
5360/5360 [==============================

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.2min finished


Epoch 1/10
6700/6700 [==============================] - 1s 195us/step - loss: 1.5325 - acc: 0.5516
Epoch 2/10
6700/6700 [==============================] - 1s 161us/step - loss: 0.5544 - acc: 0.7893
Epoch 3/10
6700/6700 [==============================] - 1s 157us/step - loss: 0.5210 - acc: 0.7916
Epoch 4/10
6700/6700 [==============================] - 1s 95us/step - loss: 0.4991 - acc: 0.7915
Epoch 5/10
6700/6700 [==============================] - 1s 99us/step - loss: 0.4851 - acc: 0.7904
Epoch 6/10
6700/6700 [==============================] - 1s 94us/step - loss: 0.4793 - acc: 0.7906
Epoch 7/10
6700/6700 [==============================] - 1s 84us/step - loss: 0.4743 - acc: 0.7910
Epoch 8/10
6700/6700 [==============================] - 1s 84us/step - loss: 0.4689 - acc: 0.7934
Epoch 9/10
6700/6700 [==============================] - 1s 95us/step - loss: 0.4647 - acc: 0.7955
Epoch 10/10
6700/6700 [==============================] - 1s 94us/step - loss: 0.4624 - acc: 0.7982


GridSearchCV(cv=5, error_score='raise',
       estimator=<tensorflow.python.keras._impl.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fc7229e99b0>,
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'optimizer': ['adam'], 'batch_size': [20, 30], 'epochs': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [30]:
grid_search.best_params_

{'batch_size': 30, 'epochs': 10, 'optimizer': 'adam'}

In [31]:
grid_search.best_estimator_